# Mount to Collab:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Imports:

In [ ]:
# Utilities
!pip install sentinelhub
!pip install speedtest-cli # Install speedtest-cli
!pip install termcolor
!pip install geopandas
!pip install xarray
!pip install cartopy

try:
    import speedtest # Import the speedtest module to check internet speed
    import os
    import pickle # to export attributes dictionary from harpconversion.py to the main collab notebook
    import datetime # for making the folder based on the dates
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    import requests
    import geopandas # to handle gdf
    from datetime import datetime # for date parameter
    from termcolor import colored # to have colored text
    from multiprocessing.pool import ThreadPool # for downloading Products in parallel
    from functools import partial # for downloading Products in parallel
    import multiprocessing # to check the number of cores in collab
    import zipfile # to extract zip files
    from glob import iglob # data access in file manager
    from os.path import join # same
    from tqdm import tqdm
    import xarray as xr
    import cartopy # improved visualizations
    import cartopy.crs as ccrs # Projected visualizations
    import cartopy.feature as cf
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
    import matplotlib.patches as mpatches
    from datetime import timedelta
    import time
    import re


    from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry)
except ModuleNotFoundError as e:
    print(colored(f'Module import error: {e.name} not found', 'red'))
else:
    print(colored('\nAll libraries properly loaded. Ready to start!!!', 'green'), '\n')


# Downloading L2 S5P data

First we Enter the Search Details:

In [ ]:
import sys
sys.path.append('/content')
from init import (
    username,
    password,
    L2_Files,
    L3_Files,
    Base_Merged_Files,
    Base_Final_Files,
    Base_PBLH_T2M_Files,
    start_date,
    end_date,
    pollutant,
    data_collection,
    aoi_file_path,
    aoi_name_to_display,
    qa,
    unit,
    num_threads,
    num_workers
)
from utils import (
    load_geojson,
    extract_aoi,
    analyze_pollutants,
    move_nc_files_and_cleanup,
    compute_lengths_and_offsets,
    get_user_choice_to_filter_data,
    filter_data,
    count_pollutant_data_types,
    get_keycloak,
    refresh_access_token,
    is_refresh_token_expired,
    is_access_token_expired,
    get_regex_pattern,
    get_variables_to_include


)

These are all the collections available:

In [ ]:
print(colored("These are all the available collections:"),'green')
for collection in DataCollection.get_available_collections():
    print(collection)

The collection we need is as follows:

In [ ]:
print(colored("The collection we need is as follows:"),'green')
DataCollection.SENTINEL5P

We can extract AOI form a geojson file using this code
(which handles polygon and multipolygon geometries):

In [ ]:
# aoi_file_path is the actual path to your GeoJSON file
geojson_path = aoi_file_path
geojson_data = load_geojson(geojson_path)

if geojson_data:
    aoi_wkt = extract_aoi(geojson_data)
    if aoi_wkt:
        # Use the extracted AOI WKT string in your code
        # For example, replacing a manual AOI string in an API request
        aoi = aoi_wkt
        print("Extracting AOI")
    else:
        print("Failed to extract AOI.")
else:
    print("Failed to load GeoJSON data.")

# In order to make the AOI format readable by the API we tweak the AOI a bit
aoi=aoi + "'"
aoi

Now we make the pollutant search:


In [ ]:
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')

# Initialize an empty list to store results dataframes
all_results = []

# Iterate through each day in the date range
current_date = start_date
while current_date <= end_date:
    # Construct the date string in the required format
    current_date_str = current_date.strftime('%Y-%m-%d')

    # Construct the query URL for the current date
    query_url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and contains(Name,'{pollutant}') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start ge {current_date_str}T00:00:00.000Z and ContentDate/Start lt {current_date_str}T23:59:59.999Z"

    # Make the request for the current date
    json = requests.get(query_url).json()

    # Check if there are any results for the current date
    if 'value' in json:
        # Convert the results to a dataframe and append to the list
        results_df = pd.DataFrame.from_dict(json['value'])
        all_results.append(results_df)

    # Move to the next day
    current_date += timedelta(days=1)

# Concatenate all dataframes in the list into a single dataframe
results = pd.concat(all_results, ignore_index=True)

# Output the final dataframe
results

We can also filter the pollutant based on its type e.g (Offline, Real-time, Reprocessed)

In [ ]:
# Call the function and get counts and total
offline_count, nrti_count, rpro_count, total_count = count_pollutant_data_types(results)

# Print the counts and total
print(f"Total number of entries: {total_count}")
print(f"Number of Offline data entries: {offline_count}")
print(f"Number of Near Real-Time data entries: {nrti_count}")
print(f"Number of Reprocessed data entries: {rpro_count}")

In [ ]:
# Get a valid user choice and the corresponding base_type
user_choice, base_type = get_user_choice_to_filter_data()

# Display the selected base_type
print(f"Base type selected: {base_type}")

# Filter the DataFrame based on the user's choice
results = filter_data(results, user_choice)

# Output or further process the filtered DataFrame
results

Check the downloading details:

In [ ]:
analyze_pollutants(results)

Code to download the data:

In [ ]:
# Setup for obtaining keycloak token and refresh token
keycloak_token, refresh_token = get_keycloak(username, password)
token_acquired_time = datetime.now()  # Store the time when the token was acquired

# Your time constraint setup remains the same
current_time = datetime.now()
time_change = pd.DateOffset(minutes=8)
expiration_time = current_time + time_change

# Get the results dataset
id_series = results['Id']

# Placeholder for file dates (You'll need to adjust how you obtain these dates)
file_dates = [start_date, end_date]  # these dates are taken from the dates which the user entered

# Find min and max dates to determine the range

min_date = min(file_dates)

max_date = max(file_dates)

# Generate folder name based on the date range
folder_name = f"{pollutant}({min_date.strftime('%B')} - {max_date.strftime('%B')}){min_date.year}"

# Define the base folder path where you want to save the files
base_folder_path = L2_Files #For drive, give the path to drive base folder here

# Combine the base folder path with the generated folder name
folder_path = os.path.join(base_folder_path, folder_name)

# Ensure the target folder exists
os.makedirs(folder_path, exist_ok=True)

# Print statements for the user
print("Total .zip files to be downloaded:",len(id_series))
print("Folder path where files will be saved:",folder_path)

In [ ]:
# Your setup to download the data
session = requests.Session()
session.headers.update({'Authorization': f'Bearer {keycloak_token}'})

max_retry_attempts = 3  # Maximum number of retry attempts for each download

for index, product_id in enumerate(id_series, start=1):  # Start counting from 1
    attempt = 0
    download_success = False

    while attempt < max_retry_attempts and not download_success:
      try:
          current_time = datetime.now()
          print(f"Attempt {attempt + 1}: Downloading {index} of {len(id_series)}: {product_id}")
          print("Current time: ", current_time)

          # Check if the refresh token has expired before attempting to refresh the access token
          if is_refresh_token_expired(token_acquired_time):
              print("Refresh token expired, re-authenticating...")
              keycloak_token, refresh_token = get_keycloak(username, password)
              token_acquired_time = datetime.now()  # Update the token acquired time
              session.headers.update({'Authorization': f'Bearer {keycloak_token}'})  # Update the session's token
              expiration_time = current_time + pd.DateOffset(minutes=8)  # Reset the expiration time for the access token
              print("New access token expiration time: ", expiration_time)
          elif current_time >= expiration_time:
              # If only the access token has expired, refresh it
              print("Refreshing access token...")
              keycloak_token = refresh_access_token(refresh_token)
              session.headers.update({'Authorization': f'Bearer {keycloak_token}'})  # Update the session's token
              expiration_time = current_time + pd.DateOffset(minutes=8)  # Reset the expiration time for the access token
              print("New access token expiration time: ", expiration_time)

          # Proceed to download the data
          url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
          response = session.get(url, allow_redirects=False)

          # Follow redirects if necessary
          while response.status_code in (301, 302, 303, 307):
              url = response.headers['Location']
              response = session.get(url, allow_redirects=True)

          # Download the file
          if response.ok:
              file_path = os.path.join(folder_path, f"{product_id}.zip")
              with open(file_path, 'wb') as file:
                  file.write(response.content)
              print(f"Successfully downloaded {product_id}")
              download_success = True
          else:
              print(f"Failed to download {product_id}. HTTP Status: {response.status_code}")
              attempt += 1
              time.sleep(5)  # Wait for 5 seconds before retrying

      except requests.exceptions.RequestException as e:
          print(f"Network error on attempt {attempt + 1} for {product_id}: {e}")
          attempt += 1
          time.sleep(5)  # Wait for 5 seconds before retrying

    if not download_success:
            print(f"Failed to download {product_id} after {max_retry_attempts} attempts.")

Code to delete corrupt files:

In [ ]:
# Define the base folder path
base_folder_path = L2_Files

# Define the minimum file size threshold in bytes (5 MB in this case)
min_file_size_bytes = 5 * 1024 * 1024  # 5 MB converted to bytes

# Initialize counters for the number of files deleted and the total files examined
deleted_files_count = 0
total_files_examined = 0

# Iterate through all files in the base folder
for folder_name, subfolders, filenames in os.walk(base_folder_path):
    for filename in filenames:
        # Construct the full file path
        file_path = os.path.join(folder_name, filename)

        # Check if the file is a .zip file
        if file_path.endswith('.zip'):
            total_files_examined += 1  # Increment the total files examined counter
            # Get the size of the file
            file_size = os.path.getsize(file_path)

            # Check if the file size is less than the minimum threshold
            if file_size < min_file_size_bytes:
                # Delete the file
                os.remove(file_path)
                deleted_files_count += 1  # Increment the deleted files counter
                print(f"Deleted {file_path} due to insufficient size.")

# Print the total number of files deleted and the total files examined
print(f"Total number of .zip files examined: {total_files_examined}")
print(f"Total number of files deleted due to being under 5MB: {deleted_files_count}")
print(f"Deleted {deleted_files_count} out of {total_files_examined} files.")

code to check if all the files have been downloaded:

In [ ]:
# Initialize an empty list to hold IDs of files that weren't downloaded
missing_files = []

for product_id in id_series:
    # Construct the expected file path for each product ID
    expected_file_path = os.path.join(folder_path, f"{product_id}.zip")

    # Check if the file exists
    if not os.path.exists(expected_file_path):
        # If the file doesn't exist, add its ID to the list of missing files
        missing_files.append(product_id)
    else:
        print(f"File for product {product_id} exists.")

# Update id_series to include only the IDs of files that weren't downloaded
id_series = missing_files

if not id_series:
    print("All files have been successfully downloaded.")
else:
    print(f"Files missing for {len(id_series)} products. Need to download again.")

In case there are missing files which need to be downloaded again, run the **"Your setup to download the data"** cell again

# Unzipping the files and keeping only NetCDF files

The following code is to extract all the zip files to get .nc files

In [ ]:
#Code to check and remount drive for connection issues
def check_and_remount_drive():
    if not os.path.ismount('/content/drive'):
        print("Drive is not mounted. Attempting to remount...")
        try:
            drive.mount('/content/drive')
        except ValueError:
            # Force unmount and remount if the mount fails
            print("Mount failed. Attempting to unmount and remount...")
            !fusermount -u /content/drive
            drive.mount('/content/drive')
    else:
        print("Drive is already mounted.")

# Function to extract zip files with retries
def extract_zip_with_retry(zip_path, extract_to, retries=3, delay=10):
    for attempt in range(retries):
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
            return True
        except zipfile.BadZipFile:
            print(colored(f'Error extracting: {zip_path} - Not a valid zip file', 'red'))
            return False
        except Exception as e:
            print(colored(f'Error on attempt {attempt + 1} - retrying in {delay} seconds: {e}', 'yellow'))
            #check_and_remount_drive()  # Remount Google Drive if necessary
            time.sleep(delay)
    return False

In [ ]:
folder_path_to_extract = folder_path
total_files_examined = 0
extracted_files_count = 0

for file in os.listdir(folder_path_to_extract):
    if file.endswith('.zip'):
        total_files_examined += 1
        zip_path = os.path.join(folder_path_to_extract, file)
        if extract_zip_with_retry(zip_path, folder_path_to_extract):
            print(colored(f'Extracted {total_files_examined}: {file}', 'green'))
            extracted_files_count += 1

print(f"Extracted {extracted_files_count} out of {total_files_examined}")

This code is to cleanup the drive folder and only keep .nc files

In [ ]:
# Specify the root directory
root_directory = folder_path_to_extract
move_nc_files_and_cleanup(root_directory)


# Variables for harp file:

In [ ]:
#make a requirments file to check package requirements
!pip freeze > requirements_for_downloading_S5P_data.txt

# Define your variables
qa_for_harp=qa
unit_for_harp=unit
aoi_for_harp=aoi_file_path
pollutant_for_harp=pollutant
L2_files_for_harp= folder_path_to_extract
# Now we create a path to a new folder for L3_Files
# Find min and max dates to determine the range
min_date = min(file_dates)
max_date = max(file_dates)
# Generate folder name based on the date range
folder_name = f"{pollutant}({min_date.strftime('%B')} - {max_date.strftime('%B')}){min_date.year}"
folder_name = folder_name.replace("L2", "L3")
# Combine the base folder path with the generated folder name
L3_files_for_harp = os.path.join(L3_Files, folder_name)
# Ensure the target folder exists
os.makedirs(L3_files_for_harp, exist_ok=True)

gdf_for_harp = geopandas.read_file(aoi_for_harp)
minx, miny, maxx, maxy = gdf_for_harp.total_bounds # Use total_bounds to get the overall bounding box

resolution = (0.01, 0.01)  #Step size for spatial re-gridding (in degrees)
# Compute offsets and number of samples using the resolution
xstep, ystep = resolution

lat_length, lat_offset, lon_length, lon_offset = compute_lengths_and_offsets(minx, miny, maxx, maxy, ystep, xstep)

# Specify the file path for variables.py
file_path = "variables_for_harp.py"

# Write the variables to variables.py
with open(file_path, "w") as f:
    f.write(f'qa_for_harp = {qa_for_harp}\n')
    f.write(f'unit_for_harp = "{unit_for_harp}"\n')
    f.write(f'aoi_for_harp = "{aoi_for_harp}"\n')
    f.write(f'pollutant_for_harp = "{pollutant_for_harp}"\n')
    f.write(f'L2_files_for_harp = "{L2_files_for_harp}"\n')
    f.write(f'L3_files_for_harp = "{L3_files_for_harp}"\n')
    f.write(f'lat_length = {lat_length}\n')
    f.write(f'lat_offset = {lat_offset}\n')
    f.write(f'lon_length = {lon_length}\n')
    f.write(f'lon_offset = {lon_offset}\n')
    f.write(f'xstep = {xstep}\n')
    f.write(f'ystep = {ystep}\n')
    #f.write(f'gdf_for_harp = {gdf_for_harp}\n')

print(colored("Variables written to variables_for_harp.py",'green'))
print(colored("Requirements for downloading data written to requierments_for_downloading_S5P_data.txt",'green'))


# Conversion from L2 to L3 using Harp

First we install Miniconda:

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
!bash /tmp/miniconda.sh -bfp /usr/local
import os
os.environ['PATH'] = "/usr/local/bin:" + os.environ['PATH']

Then we set-up the conda virtual environment

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell
conda env list
conda env remove -n myenv -y
conda create --override-channels -c conda-forge -c stcorp-forge --name myenv -y python=3.12.7
conda activate myenv
conda env list
python --version
conda install -c conda-forge harp -y # Preprocess L2 to L3 S5p data

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell
conda env list
conda activate myenv
conda env list
python --version
#sudo pip3 uninstall matplotlib
sudo apt-get install python3-matplotlib
pip install netCDF4
pip install basemap
conda install matplotlib -y
conda install cartopy -y
conda install xarray -y
conda install numpy -y
conda install pandas -y
conda install imageio -y
conda install termcolor -y
conda install conda-forge::tqdm -y
conda install anaconda::requests -y
conda install conda-forge::speedtest-cli -y
pip install geopandas -y
conda install anaconda::ipywidgets -y

Finally we check if all the necessary packages are installed:

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell
conda env list
conda activate myenv
conda env list
python --version
# use pip list and conda list to check all the packages
conda list | grep -E 'matplotlib|netCDF4|basemap|cartopy|xarray|numpy|pandas|imageio|termcolor|tqdm|requests|speedtest-cli|geopandas|ipywidgets'

Finally we run the harp.py file for L2-L3 conversion:

In [ ]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell
conda env list
conda activate myenv

python harpconversion.py

In [ ]:
from google.colab import files

# Specify the path to your .pkl file
file_path = '/content/attributes.pkl'

# Use the files.download() function to download the file to your local system
files.download(file_path)

# L3-CSV conversion

In [ ]:
# Replace 'L2' with 'L3' and assign to a new variable
pollutant_for_L3 =  pollutant.replace("L2", "L3")


# Directory containing the .nc files
L3_files_directory = L3_files_for_harp # Replace with the actual path

# Generate the regex pattern
date_regex = get_regex_pattern(base_type, pollutant_for_L3)

# List all .nc files
L3_nc_files = [f for f in os.listdir(L3_files_directory) if f.endswith('.nc')]

# Check which files do not match the regex pattern
for f in L3_nc_files:
    if not re.search(date_regex, f):
        print(f"No match: {f}")

# Filter files that match the date pattern
matching_files = [f for f in L3_nc_files if re.search(date_regex, f)]

# Extract dates from filenames and sort them
L3_nc_files_sorted = sorted(
    matching_files,
    key=lambda x: re.search(date_regex, x).group(1)
)

L3_nc_files_sorted


Then we check the total number of files we have

In [ ]:
len(L3_nc_files)


In [ ]:
len(L3_nc_files_sorted)

if the L3_nc_files are equal to the L3_nc_files_sorted then we move forward

In [ ]:
# Initialize an empty DataFrame to hold all the data
combined_data = pd.DataFrame()

In the next block we are preparing the CSV file path

In [ ]:
# Format the dates
min_date_str = min_date.strftime("%B%d")  # July01
max_date_str = max_date.strftime("%B%d")  # December31


# Construct the filename
L3_CSV_filename = f"{pollutant_for_L3}({min_date_str}-{max_date_str}).csv"

base_folder_path_L3_CSV = L3_Files #For drive, give the path to drive base folder here

# Combine the base folder path with the generated folder name
file_path_L3_CSV = os.path.join(base_folder_path_L3_CSV, L3_CSV_filename)

print(f"Path to new csv file: {file_path_L3_CSV}")

In [ ]:
for file_name in L3_nc_files_sorted:
    file_path = os.path.join(L3_files_directory, file_name)
    # Open the .nc file
    ds = xr.open_dataset(file_path)

    # Assuming you want to extract the same variables as before
    # For NO2
    #variables_to_include = ['tropospheric_NO2_column_number_density', 'cloud_fraction']
    # For CO
    #variables_to_include = ['CO_column_number_density']
    # For O3
    #variables_to_include = ['O3_column_number_density','O3_effective_temperature','cloud_fraction']
    # For Aerosol
    #variables_to_include = ['absorbing_aerosol_index']
    # For SO2
    #variables_to_include = ['SO2_column_number_density','cloud_fraction']

    variables_to_include = get_variables_to_include(pollutant_for_L3)

    df = ds[variables_to_include].to_dataframe().reset_index()

    # Extract date from file_name using the regex
    file_date = re.search(date_regex, file_name).group(1)
    # Convert string date to datetime format
    df['date'] = pd.to_datetime(file_date, format='%Y%m%d')

    # Append the extracted data to the combined DataFrame
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Once all files are processed, sort by date if not already sorted
combined_data.sort_values(by='date', inplace=True)

# Convert the time dimension if present and format it according to your needs
# Note: This part is omitted here but can be included based on the specific time formatting you require

# Export the combined dataset to CSV
combined_csv_path = file_path_L3_CSV
combined_data.to_csv(combined_csv_path, index=False)
print(f"Combined CSV file saved to: {combined_csv_path}")

# Remove NAN values from CSV using spline interpolation

You have to manually change column names here.

e.g :

1.   "tropospheric_NO2_column_number_density" for NO2
2.   "CO_column_number_density" for CO



In [ ]:
def fill_nan_values_and_flag(csv_file_path):

    # Load the CSV file
    data = pd.read_csv(csv_file_path)

    # Create a new column 'Is_Interpolated' and set it to 'No' initially
    data['Is_Interpolated'] = 'No'

    # Identify rows with NaN values in the specified column
    nan_rows_before = data['CO_column_number_density'].isna()

    # Fill NaN values using cubic spline interpolation
    data['CO_column_number_density'] = data['CO_column_number_density'].interpolate(method='spline', order=3)

    # Identify rows that were filled by checking against the original NaN flags
    nan_rows_after = data['CO_column_number_density'].isna()
    # Rows that were NaN before but not after were interpolated
    interpolated_rows = nan_rows_before & ~nan_rows_after
    data.loc[interpolated_rows, 'Is_Interpolated'] = 'Yes'

    # Group data by date and check for complete NaNs in each group
    complete_nan_dates = data.groupby('date')['CO_column_number_density'].apply(lambda x: x.isna().all())

    # Filter dates where all values are NaN
    complete_nan_dates = complete_nan_dates[complete_nan_dates].index.tolist()

    print("Dates with completely missing 'CO_column_number_density' values:")
    for date in complete_nan_dates:
        print(f"- {date}")

    # Optionally, save the filled and flagged dataset back to a CSV
    filled_csv_file_path = csv_file_path.replace('.csv', '_filled_flagged.csv')
    data.to_csv(filled_csv_file_path, index=False)

    print(f"NaN values filled using spline interpolation and flagged. Filled dataset saved to {filled_csv_file_path}.")

# Example usage
csv_file_path = file_path_L3_CSV  # Replace this with the path to your CSV file

fill_nan_values_and_flag(csv_file_path)


visualizing the spline interpolation

In [ ]:

# Load your dataset
file_path = file_path_L3_CSV

data = pd.read_csv(file_path)

# Assuming 'tropospheric_NO2_column_number_density' has NaN values you've interpolated
# Perform spline interpolation (for demonstration, using cubic spline, order=3)
data['interpolated'] = data['CO_column_number_density'].interpolate(method='spline', order=3)

# Plotting
plt.figure(figsize=(12, 6))

# Plot original data points
plt.scatter(data.index, data['CO_column_number_density'], color='blue', label='Original Data', alpha=0.5)

# Plot interpolated data
plt.plot(data.index, data['interpolated'], color='red', label='Spline Interpolation', linewidth=2)

plt.title('Visual Inspection of Spline Interpolation Fit')
plt.xlabel('Index or Time')
plt.ylabel(f'{pollutant_for_L3} column number density')
plt.legend()
plt.show()

**Tips for Visual Inspection:**

**Identify Overfitting:** Look for sections where the red spline line (interpolated data) follows the blue points (original data) too closely, including the noise. This might indicate overfitting.

**Identify Underfitting:** Notice areas where the spline does not capture significant trends or patterns in the data, suggesting underfitting.

**Adjust Order:** If you observe overfitting or underfitting, consider adjusting the order of the spline interpolation. Lower orders reduce complexity (less likely to overfit) but might miss capturing the data trend accurately (underfit). Higher orders can capture more complex patterns but risk overfitting to noise.

This process is iterative—adjust the spline order and inspect the fit visually until you find the balance that best represents your data's underlying trend.